In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import sys
from tqdm import tqdm

PATH_TO_PROJECT = os.getcwd()
FOLDR_WITH_DATASET = "DataSet"
FOLDERS_WITH_IMAGES = ["A", "N", "H", "P"]
RESULT_FILE_EXTENSION = ".txt"
PATH_TO_DATASET = os.path.join(PATH_TO_PROJECT, FOLDR_WITH_DATASET)

IMG_SIZE = 100



def create_training_all_data(FOLDERS_WITH_IMAGES):
    training_data = []
    results = []
    for folder in FOLDERS_WITH_IMAGES:
        path_to_images = os.path.join(PATH_TO_DATASET, folder)
        path_to_result = os.path.join(PATH_TO_DATASET, folder+RESULT_FILE_EXTENSION)
        list_of_results = open(path_to_result).readlines()
        list_of_results.pop(0)
        for res in list_of_results:
            splitted_res = res.split("\t")
            valency = splitted_res[1]
            arousal = splitted_res[2]
            results.append([valency, arousal])
        index = 0
        for img in tqdm(os.listdir(path_to_images)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path_to_images, img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, results[index]])  # add this to our training_data
                index += 1
            except Exception as e:
                pass
                #print("general exception", e, os.path.join(categoty_path, img))
    return training_data


training_data = create_training_all_data(FOLDSERS_WITH_IMAGES)

print(len(training_data))

100%|██████████| 121/121 [00:00<00:00, 166.00it/s]

439


In [6]:
X = []
y = []

LVLA = [1, 0, 0, 0]
LVHA = [0, 1, 0, 0]
HVLA = [0, 0, 1, 0]
HVHA = [0, 0, 0, 1]

def prepare_results(y, X, training_data):
    for features,label in training_data:
        X.append(features)
        if float(label[0]) < 50.0 and float(label[1]) < 50.0:
            y.append(LVLA)
        elif float(label[0]) < 50.0 and float(label[1]) >= 50.0:
            y.append(LVHA)
        elif float(label[0]) >= 50.0 and float(label[1]) < 50.0:
            y.append(HVLA)
        else:
            y.append(HVHA)
    X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

prepare_results(y, X, training_data)

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.models as Models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle
X = tf.keras.utils.normalize(X, axis=1)
model = Sequential()

model.add(Conv2D(2, (3, 3), input_shape=X.shape[1:], data_format='channels_last', strides=1))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(196, (3, 3), strides=1))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=3))

model.add(Conv2D(92, (3, 3), strides=1))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=3))

model.add(Dropout(0.50))
model.add(Flatten())

model.add(Dense(128))
model.add(Activation('selu'))

model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, np.array(y), batch_size=32, epochs=10, validation_split=0.2)

Train on 351 samples, validate on 88 samples
Epoch 1/10
351/351 [==============================] - 5s 15ms/sample - loss: 0.3346 - accuracy: 0.8618 - val_loss: 0.3154 - val_accuracy: 0.8977
Epoch 2/10
351/351 [==============================] - 4s 12ms/sample - loss: 0.2920 - accuracy: 0.8939 - val_loss: 0.2782 - val_accuracy: 0.8977
Epoch 3/10


In [28]:
VALIDATION_FOLDERS = ["Sn"]
validation_data = create_training_all_data(VALIDATION_FOLDERS)

X_val = []
y_val = []
prepare_results(y_val, X_val, validation_data)
X_val = tf.keras.utils.normalize(X_val, axis=1)
                
val_loss, val_acc = model.evaluate(X_val, np.array(y_val))
print(val_loss)
print(val_acc)

counter = 0
predictions = model.predict(X_val)
for i in range(len(y_val)):
    if np.argmax(predictions[i]) != np.argmax(y_val[i]):
        counter += 1
        
print(counter)

100%|██████████| 133/133 [00:00<00:00, 934.70it/s] 


133/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================